In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


## 1) 다음영화(*https://movie.daum.net/*)에서 제공하는 영화 리뷰 크롤링

In [ ]:
# 필요한 라이브러리
import requests
from bs4 import BeautifulSoup

import json
import re

import pandas as pd

In [ ]:
# 맞춤법 검사
headers = {
'User-Agent' : 'Mozilla/5.0 (Windows; U; Windows NT 6.1; ko; rv:1.9.2.8) Gecko/20100722 Firefox/3.6.8 IPMS/A640400A-14D460801A1-000000426571', # firefox UserAgent
}

def spellchecker(q):

  url = "https://m.search.naver.com/p/csearch/ocontent/spellchecker.nhn" # base url : 네이버 맞춤법 검사기

  # requests.get으로 넘겨질 파라미터 설정
  params= {
    '_callback': 'window.mycallback',
    'q': q,
  }

  response = requests.get(url,params=params, headers=headers).text
  response = response.replace(params['_callback'] + '(','') # 응답 받은 파라미터에서 callback은 제거하고 json 부분만 남김
  response = response.replace(');','')
  
  response_dict = json.loads(response)
  result_text = response_dict['message']['result']['html'] # 원하는 부분만 추출
  result_text = re.sub(r'<\/?.*?>','',result_text)

  return result_text

In [ ]:
# 반도 영화 리뷰 크롤링
pos = []
neg = []

num = 1
while num <= 50: # 1번째 페이지부터 50번째 페이지까지의 후기들을 가져옴
    source = requests.get("https://movie.daum.net/moviedb/grade?movieId=125372&type=netizen&page="+str(num)).text # 해당 사이트에 요청하여 html 소스들을 문자열(.text)로 바꿔서 source 변수에 집어넣는다. 
    soup = BeautifulSoup(source, "html.parser") # 요청해서 받은 문자열(source)을 Beautifulsoup 클래스의 생성자에게 html 소스로 파싱한다는 옵션과 함께 넘겨준다. 파싱한 결과값이 soup 변수에 저장된다. 
    hotKeys = soup.select("em.emph_grade") # 별점 값을 얻어오기 위해 html을 파싱한 soup 변수 내부에서 em 메타의 class명이 emph_grade인 모든 문자열을 select. select는 해당 키워드들을 리스트 형태로 반환한다.
    reviews = soup.select("p.desc_review") # 리뷰 내용을 얻어오기 위해 html을 파싱한 soup 변수 내부에서 p 메타의 class명이 desc_review인 모든 문자열을 select
    
    for key, review in zip(hotKeys, reviews):
        if int(key.text) <= 5: # 5점 이하인 리뷰들만 부정 리뷰로 가져옴
            if review.text.strip() != "" and review.text.strip() != ".": # strip() : 문자열의 앞, 뒤의 연속된 공백 제거
                text = review.text.strip().replace("\r", " ") # 줄바꿈을 의미하는 \r 문자는 공백 한 칸으로 대체
                text = spellchecker(text) # 맞춤법 검사기 적용
                temp = [text]
                temp.append(0) # 부정(0) Labeling
                neg.append(temp) # 부정 list에 추가
        elif int(key.text) >= 7: # 7점 이상인 리뷰들만 긍정 리뷰로 가져옴
            if review.text.strip() != "" and review.text.strip() != ".":
                text = review.text.strip().replace("\r", " ")
                text = spellchecker(text)
                temp = [text]
                temp.append(1) # 긍정(1) Labeling
                pos.append(temp) # 긍정 list에 추가
        
    num = num + 1

In [ ]:
# 코코 영화 리뷰 크롤링
num = 1
while num <= 30: 
    source = requests.get("https://movie.daum.net/moviedb/grade?movieId=110929&type=netizen&page="+str(num)).text
    soup = BeautifulSoup(source, "html.parser")
    hotKeys = soup.select("em.emph_grade")
    reviews = soup.select("p.desc_review")
    
    for key, review in zip(hotKeys, reviews):
        if int(key.text) <= 4:
            if review.text.strip() != "" and review.text.strip() != ".":
                text = review.text.strip().replace("\r", " ")
                text = spellchecker(text)
                temp = [text]
                temp.append(0)
                neg.append(temp)
        elif int(key.text) >= 7:
            if review.text.strip() != "" and review.text.strip() != ".":
                text = review.text.strip().replace("\r", " ")
                text = spellchecker(text)
                temp = [text]
                temp.append(1)
                pos.append(temp)
        
    num = num + 1

In [ ]:
reviews = pos + neg # 긍정 리뷰와 부정 리뷰를 하나의 list로 합침 
df = pd.DataFrame(reviews, columns=['Review', 'Label']) # dataframe 생성
df.to_csv('/gdrive/My Drive/공모전_국어정보처리/daum_movie_reviews.csv',index=False, encoding="utf-8") # csv파일로 저장

In [ ]:
df

,Review,Label
0,최근에 본 영화 중 가장 재미있어요!!,1
1,다들 영화 전문가 되셨네. 난 괜찮았음.,1
2,다 좋은데 전투 중에 감상적인 장면은 최대한 짧게 가자 적군이 기다려주냐 흐름 확 ...,1
3,괜찮아요 ^^ 평점은 별로라서 기대 안 했어요 ~~의외로 재밌게 봤어요,1
4,솔직히 볼만했음,1
...,...,...
632,이 영화의 세계관이 부산행 그 후가 아닌 분노의 질주 매드맥스 인가 봅니다. 배...,0
633,좀비물이면 좀 여기에 집중해야지... 이건 머 레이싱 영화인지 악당과의 총싸움인지....,0
634,이랑보다 재미없더라 근데 380만 명이나 봤더라,0
635,내가 완악한 것인지 아역배우들의 눈물에 한 개도 공감이 안가네. ㅠㅠ 뭐가 슬퍼?,0


##2) 대회 제공 데이터(Naver sentiment movie corpus v1.0) 샘플링

In [ ]:
file = pd.read_csv('/gdrive/My Drive/공모전_국어정보처리/ratings.txt', delimiter = '\t')

del file['id']
file = file.dropna() # missing values 삭제
file.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 199992 entries, 0 to 199999
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   document  199992 non-null  object
 1   label     199992 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 4.6+ MB


In [ ]:
# 긍정 리뷰 150개, 부정 리뷰 150개 샘플링
file1 = file.iloc[10000:10150,:]
file2 = file.iloc[150000:150150,:]
data = pd.concat([file1, file2])
data.index = pd.RangeIndex(start=1, stop=len(data) + 1, step=1)
data

,document,label
1,진정한 사랑입니다.,1
2,볼만한데..,1
3,나름 시간보내기용으로는 쵝오,1
4,작가는 무엇을 해야하는가,1
5,완전재미있어요. 또 보고 싶어요,1
...,...,...
296,수많은 늑대인간 영화와는 다른 시도.,0
297,러시아와 조지아전쟁이 벌어진원인이 누군지생각하고 만들었나? 가해자가 피해자로 둔갑하...,0
298,이런거 보느니 차라리 정통 포르노를 보고말지..,0
299,너무 아프다.,0


In [ ]:
# 맞춤법 검사기 적용
reviews = []
for review, label in zip(data.loc[:,'document'], data.loc[:,'label']):
  review = spellchecker(review)
  temp = [review]
  temp.append(label)
  reviews.append(temp)

In [ ]:
df = pd.DataFrame(reviews, columns=['Review', 'Label']) # dataframe 생성
df.to_csv('/gdrive/My Drive/공모전_국어정보처리/naver_reviews.csv',index=False, encoding="utf-8") # csv파일로 저장

In [ ]:
df

,Review,Label
0,진정한 사랑입니다.,1
1,볼만한데..,1
2,나름 시간 보내기용으로는 최고,1
3,작가는 무엇을 해야 하는가,1
4,완전 재미있어요. 또 보고 싶어요,1
...,...,...
295,수많은 늑대인간 영화와는 다른 시도.,0
296,러시아와 조지아 전쟁이 벌어진 원인이 누군지 생각하고 만들었나? 가해자가 피해자로 ...,0
297,이런 거 보느니 차라리 정통 포르노를 보고 말지..,0
298,너무 아프다.,0


## 3) Youtube TV프로그램 영상 댓글 크롤링

In [ ]:
# 필요한 라이브러리
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver as wd

In [ ]:
## 시간 측정 - 시작 시간
start = time.time()      

# 크롬 드라이버 실행
driver = wd.Chrome(executable_path=r"C:/Users/jeram/Downloads/chromedriver")

# 크롤링할 유튜브 URL 입력
url = "https://www.youtube.com/watch?v=Og_X5gTXTF0"
driver.get(url)

# 크롤링할 유튜브 URL이 크롬에서 열리면 지금까지 로딩된 해당 페이지의 전체 높이를 가져옴
page_height = driver.execute_script("return document.documentElement.scrollHeight")

while True:
    # 높이만큼 맨 밑까지 스크롤을 내린다.
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    
    # 맨 아래까지 스크롤을 내렸을 때 다시 로딩되는 댓글을 1초 기다린다.
    time.sleep(1.0)
    
    # 다시 지금까지 로딩된 페이지의 전체 높이를 가져온다.
    new_page_height = driver.execute_script("return document.documentElement.scrollHeight")

    # 로딩이 끝까지 되었다면 이전에 저장한 높이와 새로 추가된 높이가 동일할 것이다.
    if new_page_height == page_height:
        break # 그러면 크롤링을 중지한다.
    
    # 로딩이 덜 끝났다면 새로 추가된 높이만큼 업데이트해주고 다시 반복
    page_height = new_page_height

# 크롤링이 끝났다면 해당 페이지의 html source를 저장하고 크롬드라이버 close
html_source = driver.page_source
driver.close()

In [ ]:
# BeautifulSoup 라이브러리를 이용하여 html에서 댓글만 가져오기
bf = BeautifulSoup(html_source, "lxml")

# 크롬 개발자 도구를 확인해서 ID와 댓글의 위치를 찾아 select() 함수를 이용하여 저장
youtube_user_IDs = bf.select("div#header-author > a > span")
youtube_comments = bf.select("yt-formatted-string#content-text")

In [ ]:
# USER ID 배열
str_youtube_userIDs = []

# USER 댓글 내용 배열
str_youtube_comments = []  

# 불필요한 공백이나 탭은 모두 제거
for i in range(len(youtube_user_IDs)):
    str_tmp = str(youtube_user_IDs[i].text)
    str_tmp = str_tmp.replace('\n', '')
    str_tmp = str_tmp.replace('\t', '')
    str_tmp = str_tmp.replace('   ','')
    str_youtube_userIDs.append(str_tmp)

    str_tmp = str(youtube_comments[i].text)
    str_tmp = str_tmp.replace('\n', '')
    str_tmp = str_tmp.replace('\t', '')
    str_tmp = str_tmp.replace('   ','')
    str_youtube_comments.append(str_tmp)

In [ ]:
## 데이터프레임의 형태로 저장한다.
pd_data = {"comment":str_youtube_comments}
youtube_pd = pd.DataFrame(pd_data)

In [ ]:
## 데이터프레임을 csv 파일로 내보낸다.
youtube_pd.to_csv("C:/Users/jeram/Downloads/Queendom_GIDLE_comments.csv")

# 전체 수행 시간 출력
print("Running Time : ", time.time() - start)